#### Imports

In [ ]:
import pandas as pd
import matplotlib
import numpy as np
import pypsa
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
import math
import itertools
import statsmodels.api as sm

from utils import market_values, market_values_links, market_values_by_time_index, market_values_links_con, congestion_rent_link, convert_ISO_3166_2_to_1, generation, generation_links, market_values_storage_units, generation_storage_units

# general variables
font1 = {'fontname':'Calibri'}
PLOT_DIR = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/'
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

In [ ]:
n =pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")

In [ ]:
# sort carriers by similar technology / underlying energy for better investigation
car_bus_el = ['AC','battery','Li ion','low voltage','home battery' ]
car_bus_heat = ['residential rural heat', 'services rural heat','residential urban decentral heat', 'services urban decentral heat', 'urban central heat']
car_bus_bio_gas = ['H2','biogas','solid biomass','solid biomass for industry','gas for industry','H2 liquid']

car_gen_vre = ['offwind-ac', 'offwind-dc','onwind', 'solar', 'ror', 'solar rooftop']
car_gen_soltherm = [ 'residential rural solar thermal', 'services rural solar thermal', 'residential urban decentral solar thermal', 'services urban decentral solar thermal','urban central solar thermal',]

car_link_imp1 = ['OCGT', 'H2 Electrolysis', 'H2 Fuel Cell', 'Sabatier', 'SMR', "Fischer-Tropsch", "urban central gas CHP", "battery charger", "battery discharger"]
car_link_imp2 = ["OCGT", "H2 Electrolysis", "Fischer-Tropsch","Sabatier", "H2 Fuel Cell", "urban central gas CHP", "urban central gas CHP CC", "battery charger", "battery discharger", "home battery charger", "home battery discharger"   ]
car_link_mix = ['OCGT', 'H2 Electrolysis', 'H2 Fuel Cell', 'Sabatier', 'SMR', "Fischer-Tropsch", "urban central gas CHP", "battery charger", "battery discharger", "residential rural resistive heater","residential rural ground heat pump","residential rural gas boiler"]
car_link_resisitve_heater = ["residential rural resistive heater", "services rural resistive heater", "residential urban decentral resistive heater", "services urban decentral resistive heater", "urban central resistive heater" ]
car_link_gas_boiler = ["residential rural gas boiler", "services rural gas boiler", "residential urban decentral gas boiler", "services urban decentral gas boiler", "urban central gas boiler"]
car_link_heat_pump = ["residential rural ground heat pump", "services rural ground heat pump", "residential urban decentral air heat pump", "services urban decentral air heat pump", "urban central air heat pump"  ]

car_imp_fneum = ["onwind", "solar", "offwind-ac", "solar rooftop", "OCGT", "H2 Electrolysis", "H2 Fuel Cell", "battery charger", "battery discharger", "V2G", "urban central air heat pump",  "urban central resistive heater", "urban central gas boiler", "urban central gas CHP", "urban central gas CHP CC", "Fischer-Tropsch", "PHS"]
car_imp2_fneum = ["ror", "SMR", "SMR CC", "home battery charger", "home battery discharger"]
car_imp3_fneum = ["gas", "urban central solar thermal"]

# TOOD: detect similar energy carriers; combine similar energy carriers: offwind(ac+dc), "solar(+rooftop)", battery charger(+home), battery discharger(+home), heat pump (all), resisitive eater (all), gas boiler (all)
# within [residential rural, services rural, residential urban decentral, services urban decentra, urban central] concentrate on urban central

# carrier buses
car_bus_imp = ["AC", "H2", "battery", "Li ion", "residential rural heat", "urban central heat", "biogas", "solid biomass", "H2 liquid", "low voltage", "home battery" ]

#### General Investigation

key data
- 365 * 24 / 3 = 2920 snapshots over one year (every 3 hours) -> rows
- 3086 different buses
- 181 different locations (+ EU: 182)
- 1707 different generators
- 8375 links
- 1635 stores
- 170 storage units
- no hydrogen network
- no network expansion (from current state)

In [ ]:
# network plot
m = n.copy()
m.mremove("Bus",m.buses[n.buses.x == 0].index )

fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.EqualEarth()}, figsize=(8, 8))

bus_locs = pd.Index(m.buses.location.unique())
load_distribution = m.loads_t.p[bus_locs].sum()/m.loads_t.p[bus_locs].sum().max()

m.plot(bus_sizes=1*load_distribution, ax=ax, title="Load distribution", projection=ccrs.EqualEarth())
plt.show()

### Spatial distribution of market values, generation and  per technology

#### Regions

In [ ]:
onshore_regions['coords'] = onshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
onshore_regions['coords'] = [coords[0] for coords in onshore_regions['coords']]
onshore_regions["name"] = onshore_regions.index
offshore_regions['coords'] = offshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
offshore_regions['coords'] = [coords[0] for coords in offshore_regions['coords']]
offshore_regions["name"] = offshore_regions.index

In [ ]:
# onshore and offshore regions on map (overview)
fig = plt.figure(figsize=(32, 26))
crs = ccrs.PlateCarree()

ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')
ax.set_title("onshore and offshore regions", fontsize=16, **font1)

onshore_regions.to_crs(crs.proj4_init).plot(ax=ax,
                     linewidth=0.2,
                     facecolor="white",
                     edgecolor = 'grey')

# offshore_regions.to_crs(crs.proj4_init).plot(ax=ax,
#                      linewidth=0.2,
#                      facecolor="white",
#                      edgecolor = 'grey')

for idx, row in onshore_regions.iterrows():
    plt.annotate(text=row['name'], xy=row['coords'],
                 horizontalalignment='center')

# for idx, row in offshore_regions.iterrows():
#     plt.annotate(text=row['name'], xy=row['coords'],
#                  horizontalalignment='center')

plt.show()
# plt.close()
# fig.savefig(f"{PLOT_DIR}01_general/regions.png")

#### Market value and generation per region

In [ ]:
# calc market values, generation, lmps for generators, links and storage units
df_regions_onshore = onshore_regions.copy()
df_regions_offshore = offshore_regions.copy()

# function for carriers in n.generators.carrier.unique() #13
for carrier in n.generators.carrier.unique():
    df_regions_onshore[f"{carrier}_mv"] = market_values(n, carrier)
    df_regions_offshore[f"{carrier}_mv"] = market_values(n, carrier)
    # generation in TWh
    df_regions_onshore[f"{carrier}_gen"] = generation(n, carrier) / 1000
    df_regions_offshore[f"{carrier}_gen"] = generation(n, carrier) / 1000
    # lmps
    df_regions_onshore[f"{carrier}_gen"] = generation(n, carrier) / 1000
    df_regions_offshore[f"{carrier}_gen"] = generation(n, carrier) / 1000

# function for carriers in n.links.carrier.unique() #53
for carrier in n.links.carrier.unique():
    df_regions_onshore[f"{carrier}_mv"] = market_values_links(n, carrier)
    df_regions_onshore[f"{carrier}_gen"] = generation_links(n, carrier) / 1000

# function for carriers in n.storage_units.carrier.unique() #2
for carrier in n.storage_units.carrier.unique():
    df_regions_onshore[f"{carrier}_mv"] = market_values_storage_units(n, carrier)
    df_regions_onshore[f"{carrier}_gen"] = generation_storage_units(n, carrier) / 1000

# set market values to nan where generation in corresponding region is lower than % quantile
qt = 0.2
for carrier in (n.generators.carrier.unique().tolist() +
                n.links.carrier.unique().tolist() +
                n.storage_units.carrier.unique().tolist()):
    index = df_regions_onshore[f"{carrier}_gen"] <= np.quantile(df_regions_onshore[f"{carrier}_gen"], qt)
    df_regions_onshore[f"{carrier}_mv"][index] = np.nan

# calc lmps at the buses (lmps that are only present for EU (e.g. oil) are nan at the moment)
# TODO: decide if EU lmps are used as lmp for all regions
for carrier_bus in n.buses.carrier.unique():
    # index would be same names as the bus (not the location as it is in the index of
    # df_regions_onshore so far -> map location to make sure the right lmp is set
    locs = n.buses.location[n.buses[n.buses.carrier == carrier_bus].index]
    lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier == carrier_bus].index].mean()
    df = pd.concat([lmps, locs], axis=1).rename(columns={0: f"{carrier_bus}_lmp"})
    df.set_index("location", inplace=True)
    if df.size == 1:
        if df.index == "EU":
            df = pd.DataFrame(np.repeat(df.values, 181), index=df_regions_onshore.index, columns=[f"{carrier_bus}_lmp"])
            df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]
    else:
        df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]

df_regions_onshore.head(3)
# df_regions_offshore.head(3)

In [ ]:
# carrier mv and generation plot
carrier = "solar" # ""onwind" # "ror" # "onwind"
labels=[f"{carrier}_mv", f"{carrier}_gen"]

fig, axs = plt.subplots(ncols=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(16, 14))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs):
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    if i==0:
        title = f"{carrier} market value"
        label = "€/MWh"
    elif i==1:
        title = f"{carrier} generation"
        label = "TWh"

    df_regions_onshore.to_crs(crs.proj4_init).plot(column=labels[i],
                                                   ax=ax,
                                                   cmap=plt.get_cmap("magma_r"),
                                                   linewidth=0.05,
                                                   edgecolor = 'grey',
                                                   legend=True,
                                                   legend_kwds={'label':label,
                            'orientation': "vertical",
                                      'shrink' : 0.4})

    ax.set_title(title, fontsize=16, **font1)
plt.show()

In [ ]:
# compare two carriers market value plot
labels=["onwind_mv", "solar_mv"]

fig, axs = plt.subplots(ncols=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(16, 14))
crs = ccrs.EqualEarth()
vmin = min(np.nanmin(df_regions_onshore[labels[0]]), np.nanmin(df_regions_onshore[labels[1]]))
vmax = max(np.nanmax(df_regions_onshore[labels[0]]), np.nanmax(df_regions_onshore[labels[1]]))

for i, ax in enumerate(axs):
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')
    ax.set_title(f"{labels[i][:-3]} market value", fontsize=16, **font1)

    df_regions_onshore.to_crs(crs.proj4_init).plot(column=labels[i],
                                                   ax=ax,
                                                   cmap=plt.get_cmap("magma_r"),
                                                   vmin=vmin,
                                                   vmax=vmax,
                                                   linewidth=0.05,
                                                   edgecolor = 'grey',
                                                   legend=True,
                                                   legend_kwds={'label':"€/MWh",
                            'orientation': "vertical",
                                      'shrink' : 0.4})


plt.show()

In [ ]:
df_regions_onshore[labels[0]].sort_values(ascending=False)

__Findings__
- __ror:__ highest mv in BEL, high in DEU, HRV (Croatia) and LUX; no data in NOR, little in ESP and FRA | highest production in ITA, CHE & AUT; interesting that there is no data for NOR
- __onwind:__ very high mv in a lot of parts of ITA, general higher values in eastern europe and at the coastline of the adriatic sea and lower at the atlantic coast, rest of the mediterranean Sea and baltic sea | highest production in IRL, very high in DNK and GBR; general hig at the coastline of the atlantic and low at the rest
- __solar:__ highest mv in BEL, NLD & GBR; general small range of mv troughout europe | highest production at the mediterranean Sea (ESP, GRC)
- __solar rooftop:__ highest mv in BEL NLD; general decreasing values from north to south | highest production in parts of FRA & ESP; increasing production from north to south
- __H2 Electrolysis:__ no very clear pattern; highest values in BEL, NLD and NOR; Tendency of higher values on iberian peninsula, scandinavia and inner europe
- __H2 Fuel Cell:__ highest values for central europe (DEU, BEL,NLD)
- __OCGT:__ very high values in POL, LTU, LVA & DNK; high values in DEU, iberian peninsula, UK & south of sweden
- __battery charger:__ highest values in central europe (BEL, NLD, DEU, AUT); high prices in eastern europe; moderate prices in scandinavia; low prices on iberian peninsula
- __battery discharger:__ similar pattern as battery charger with higher values in FIN and scnadinavia beeing cheaper than iberain peninsula
- __home battery charger:__ similar pattern as battery charger with a slightly higher magnitude
- __home battery discharger:__ same pattern like home battery charger with a slightly higher magnitude
- heat pumps (air-sourced/ground-sourced)
- __urban central gas boiler:__ by distance highest mv in one part of NOR; except from that very similar mv (range: 120 -150 €/MWh) in all otther regions with some exceptions
- __residential rural resistive heater:__ highes mv for ROU (Romania) & BGR (Bulgaria); general very high in the south-east of europe (GRC, SRB (Serbia), MKD (north Macedonia)
- __services rural resistive heater:__ same pattern as residential rural resistive heater with lower magnitude of mvs | "residential urban decentral resistive heater" and "services urban decentral resistive heater" are also similar
- __hydro:__ low mv in scandinavia, high mv in souther europe especially high in south-east europe (HUN, ROU, BGR); high generation in scandinavia especially south of NOR, generally higher generation furhter to the mediteraanean sea, almost nor generation in north of FRA & DEU and Benelux
- __PHS:__ no clear pattern in mv, only 72 regions across europa have values; generation is high in CHE, AUT & ESP
- __Fischer-Tropsch:__ Calculation unclear (bus1 is always EU with a constant oil lmp) -> Fischer-Tropsch und Sabatier haben regional keine unterschiedlichen
Marktwerte, da regional aggregiert.
- __Sabatier:__ Calculation unclear (bus1 is always EU with a constant lmp) -> Fischer-Tropsch und Sabatier haben regional keine unterschiedlichen
Marktwerte, da regional aggregiert.


In [ ]:
# offshore regions
carrier = "offwind-ac"
labels=["offwind-ac_mv", "offwind-dc_mv" ] # f"{carrier}_gen"

fig, axs = plt.subplots(ncols=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(16, 14))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs):
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')
    ax.set_title(f"{labels[i][:-3]} market value", fontsize=16, **font1)

    df_regions_offshore.to_crs(crs.proj4_init).plot(column=labels[i],
                                                    ax=ax,
                                                    cmap=plt.get_cmap("magma_r"),
                                                    linewidth=0.05,
                                                    edgecolor = 'grey',
                                                    legend=True,
                                                    legend_kwds={'label':"€/MWh",
                            'orientation': "vertical",
                                      'shrink' : 0.4})
plt.show()

In [ ]:
df_regions_offshore[labels[1]].sort_values(ascending=False)

__Findings__
- __offwind-ac:__ highest mv in ITA, HRO (Croatia); low around UK & at the coastlin of NOR & iside the baltic | high production at the norther coastline of FRA, BEL, NLD & DEU; southcoast of GBR and eastcoast of ROU (Romania) & BGR (Bulgaria) (Black sea)
- __offwind-dc:__ highest mv in ITA, SVN (Slowenia) & MNE (Montenegro) | high production at the norther coastline of BEL, NLD & DEU; southcoast of GBR, westcoast of FIN & GRC| mv of offwind-ac and offwinf-dc are rather similar; generation pattern is quite similar with more generation for offwind-dc

#### DIff(LMP-MV) per region

In [ ]:
# get relevant carrier of bus where carrier leads to (generators & links) manually
carrier = "H2 Electrolysis"
print(f"generator: {n.buses.loc[n.generators[n.generators.carrier == carrier]['bus']]['carrier'].unique()}")
print(f"link: {n.buses.loc[n.links[n.links.carrier == carrier]['bus1']]['carrier'].unique()}")
print(f"storage_unit: {n.buses.loc[n.storage_units[n.storage_units.carrier == carrier]['bus']]['carrier'].unique()}")

In [ ]:
# create mapping with carrier and carrier bus:
all_carriers = n.generators.carrier.unique().tolist() + n.links.carrier.unique().tolist() + n.storage_units.carrier.unique().tolist()

all_carriers_buses = []

for carrier in all_carriers:
    gen_bus = carrier_bus = n.buses.loc[n.generators[n.generators.carrier == carrier]['bus']]['carrier'].unique().tolist()
    link_bus = n.buses.loc[n.links[n.links.carrier == carrier]['bus1']]['carrier'].unique().tolist()
    su_bus = n.buses.loc[n.storage_units[n.storage_units.carrier == carrier]['bus']]['carrier'].unique().tolist()
    all_carriers_buses.append((gen_bus + link_bus + su_bus)[0])

map_carrier_bus = dict(zip(all_carriers, all_carriers_buses))

In [ ]:
# calc difference of carriers and add to df_onshore_regions
for carrier in all_carriers:
    carrier_bus = map_carrier_bus[carrier]
    df_regions_onshore[f"diff({carrier}_mv,{carrier_bus}_lmp)"] = (df_regions_onshore[f"{carrier}_mv"] - df_regions_onshore[f"{carrier_bus}_lmp"])

df_regions_onshore.head(3)

In [ ]:
carrier = "H2 Electrolysis"
carrier_bus = map_carrier_bus[carrier]

fig = plt.figure(figsize=(12, 8))
crs = ccrs.EqualEarth()

ax = plt.axes(projection=ccrs.EqualEarth())
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')
ax.set_title(f"diff(lmp({carrier_bus})-mv({carrier})", fontsize=16, **font1)

df_regions_onshore.to_crs(crs.proj4_init).plot(column=f"diff({carrier}_mv,{carrier_bus}_lmp)",
                     ax=ax,
                     cmap=plt.get_cmap("magma_r"),
                     linewidth=0.05,
                     edgecolor = 'grey',
                     legend=True,
                     legend_kwds={'label':"€/MWh",
                        'orientation': "vertical"})

plt.show()

In [ ]:
df_regions_onshore[f"diff({carrier}_mv,{carrier_bus}_lmp)"].dropna().sort_values(ascending=False)

__Findings:__

TODO: This has to be rewritten as the order changed from lmp-mv to mv-lmp!!!!!!!

- __Implications:__ high positive values mean that the lmp at the bus is on average (weighted by the production) higher than the mv of the technology which means the technology produces at times of lmps that are lower than the average (bad for the technology); high negative values mean that the technology produces disproportionately often if the lmp is relatively high (good for the technology); mv is just a weighted average of the lmp weighted by the generation, the lmp value in this case is just the average lmp over all timesteps
- __["H2","H2 Electrolysis"]:__ high diff in middle and middle-east europe (DEU, ITA, CZE, LUX, ...)
- __["battery","battery charger"]:__ very high diff in middle europe; low diff in NOR and SWE
- __["home battery","home battery charger"]:__ similar to ["battery","battery charger"]
- __["AC","solar"]:__ range(0,60); high in middle and middle-east europa low in scandinavia, UK
- __["AC","onwind"]:__ range(-30,47); high negative values in ITA
- __["AC","offwind-ac"]:__ range(-50,40); high negative values in ITA & adriatic sea, highest positive values in NLD
- __["low voltage","solar rooftop"]:__ range(-5,60); slightly negative values in GBR; verhy high values in middle and middle-east europe (DEU,ROU,HRV)

#### Value Factor Analysis

In [ ]:
# calc value factor for carriers
for carrier in all_carriers:
    df_regions_onshore[f"{carrier}_vf"] = df_regions_onshore[f"{carrier}_mv"] / df_regions_onshore[f"{map_carrier_bus[carrier]}_lmp"]

In [ ]:
carrier = "H2 Electrolysis"

fig = plt.figure(figsize=(12, 8))
crs = ccrs.EqualEarth()

ax = plt.axes(projection=ccrs.EqualEarth())
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')
ax.set_title(f"{carrier} value factors", fontsize=16, **font1)

df_regions_onshore.to_crs(crs.proj4_init).plot(column=f"{carrier}_vf",
                     ax=ax,
                     cmap=plt.get_cmap("magma_r"),
                     linewidth=0.05,
                     edgecolor = 'grey',
                     legend=True,
                     legend_kwds={'label':"€/MWh",
                        'orientation': "vertical"})

plt.show()

In [ ]:
# Scatterplot value factor against market share (generation)

# calc market share per technology
for carrier_bus in ["AC", "low voltage", "H2", "battery", "urban central heat", "oil"]:
    # get all technologies that have same bus
    carriers = np.array(list(map_carrier_bus.keys()))[[i for i, x in enumerate(list(map_carrier_bus.values())) if x == carrier_bus]].tolist()

    # calc share of generation
    for carrier in carriers:
        df_regions_onshore[f"{carrier}_gen_share"] = df_regions_onshore[f"{carrier}_gen"] / df_regions_onshore[[s + "_gen" for s in carriers]].sum(axis=1)

In [ ]:
# carrier = "V2G" # "H2 Fuel Cell" # "urban central gas CHP" # "urban central gas CHP CC" # "OCGT"

# no data for Fischer-Tropsch
for carrier in [x for x in car_imp_fneum if x != 'Fischer-Tropsch']:

    x = df_regions_onshore[f"{carrier}_gen_share"]
    y = df_regions_onshore[f"{carrier}_vf"]
    x = x[x.notna() & y.notna()]
    y = y[x.notna() & y.notna()]

    fig = plt.figure(figsize=(10, 6))
    p = np.poly1d(np.polyfit(x, y, 1))
    t = np.linspace(start = 0, stop = math.ceil(max(x)), num = 1000)
    plt.plot(x, y, 'o', label=carrier)
    plt.plot(t, p(t), '-')
    plt.title("Scatter: value factor vs. generation share")
    plt.xlabel("generation share (%)")
    plt.ylabel("value factor (%)")
    plt.legend()
    plt.show()

In [ ]:
# comparing plot:
carriers = car_imp_fneum.copy()
carriers = [e for e in carriers if e not in ["OCGT", "Fischer-Tropsch", "urban central gas CHP",  "urban central gas CHP CC", "H2 Fuel Cell", "V2G"]]
colors = matplotlib.cm.tab20(range(len(carriers)))
markers = itertools.cycle((',', '+', '.', 'o', '*'))

plt.figure(figsize=(10, 6))

for carrier, i in zip(carriers, np.arange(0,len(carriers)-1)):
    x = df_regions_onshore[f"{carrier}_gen_share"]
    y = df_regions_onshore[f"{carrier}_vf"]
    x = x[x.notna() & y.notna()]
    y = y[x.notna() & y.notna()]

    p = np.poly1d(np.polyfit(x, y, 1))
    t = np.linspace(start = 0, stop = 1, num = 5)
    plt.plot(t, p(t), '--', label=carrier, color=colors[i], marker=next(markers))

plt.title("Trend: Value factor vs. generation share")
plt.xlabel("generation share (%)")
plt.ylabel("value factor (%)")
#plt.ylim([0,2])
plt.legend(loc='lower left', ncol=2)
plt.show()

#### Market values per country

In [ ]:
# geopandas country data
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.drop(["pop_est","gdp_md_est"], axis=1)
# Remove French Guiana from France.
shape = world[world['name'] == 'France']['geometry'].all()
fr_df = pd.Series(['France', 'France'], name='country')
fr_df = gpd.GeoDataFrame(fr_df, geometry=[shape[1], shape[2]])
fr_df = fr_df.dissolve(by='country')
world.geometry[world['name'] == 'France'] = fr_df['geometry'].values
# Remove Svalbard
shape = world[world['name'] == 'Norway']['geometry'].all()
no_df = pd.Series(['Norway'], name='country')
no_df = gpd.GeoDataFrame(no_df, geometry=[shape[1]])
no_df = no_df.dissolve(by='country')
world.geometry[world['name'] == 'Norway'] = no_df['geometry'].values

In [ ]:
# convert country codes and map
mv_country_onshore = df_regions_onshore.drop(["geometry", "coords"], axis=1)
mv_country_onshore["country"] = mv_country_onshore.index.str[:2]
mv_country_onshore = mv_country_onshore.groupby(by="country").mean()
mv_country_onshore["iso_a3"] = pd.Index([convert_ISO_3166_2_to_1[cc] for cc in mv_country_onshore.index])
mv_country_onshore=pd.merge(world, mv_country_onshore,on='iso_a3')

mv_country_onshore['coords'] = mv_country_onshore['geometry'].apply(lambda x: x.representative_point().coords[:])
mv_country_onshore['coords'] = [coords[0] for coords in mv_country_onshore['coords']]
mv_country_onshore.head()

In [ ]:
# compare market value and generation
labels=["solar_mv", "solar_gen"]

fig, axs = plt.subplots(ncols=2, subplot_kw={'projection': ccrs.PlateCarree()},
                        figsize=(16, 14))
crs = ccrs.PlateCarree()

for i, ax in enumerate(axs):

    if i == 0:
        title=f"{labels[i][:-3]} (market value)"
        label="€/MWh"
    else:
        title=f"{labels[i][:-4]} (generation)"
        label="TWh"

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')
    ax.set_title(title, fontsize=16, **font1)

    mv_country_onshore.to_crs(crs.proj4_init).plot(column=labels[i],
                         ax=ax,
                         cmap=plt.get_cmap("magma_r"),
                         linewidth=0.05,
                         edgecolor = 'grey',
                         legend=True,
                         legend_kwds={'label': label,'orientation': "vertical",'shrink' : 0.4}
                                                   )

    for idx, row in mv_country_onshore.iterrows():
        ax.annotate(text=row['iso_a3'], xy=row['coords'], color='white',
                     bbox=dict(boxstyle='round,pad=0.2', fc='grey', alpha=0.3),
                     horizontalalignment='center')

plt.show()

In [ ]:
# carriers for which market_values function does not work
# -> there are 24 carrier which are not a generators but attached to a bus; from the generators only 2 are also attached to buses {'gas', 'oil'}
# all renewables are produced at generators and are not directly attached to a bus; the generator is attached to the bus (thats also why they do not have lmps)
# set(n.buses.carrier.tolist()) - set(n.generators.carrier.tolist()) # 24

In [ ]:
# fig.savefig(f"{PLOT_DIR}mv_country.png")

In [ ]:
mv_country_onshore.isna().sum()

In [ ]:
# market value of selected carriers per region
carriers_h = ['OCGT','urban central gas CHP','SMR']
carriers_l = ['H2 Electrolysis','onwind', 'solar','ror']
carriers_comp = ['solar', 'solar rooftop', 'H2 Fuel Cell', 'SMR', 'home battery charger']

ax = mv_country_onshore[[c + "_mv" for c in carriers_comp]].plot(figsize=(18,8), style="--", marker='o', grid=True)
plt.legend(loc='upper left', ncol=3, prop={'size': 12})
ax.set_xticks(range(len(mv_country_onshore)))
ax.set_facecolor('white')
ax.set_axisbelow(True)
plt.grid(color='grey', alpha=0.5)
ticks = ax.set_xticklabels(labels = [item for item in mv_country_onshore.iso_a3.tolist()], rotation=0)
# Set the background color to white

### Temporal distribution of market values (Seasonality)

In [ ]:
#TODO: plots to show the seasonality (winter, summer) (quarters)

### Spatial and temporal distribution of prices:

- electricity
- heat (various subcategories)
- hydrogen

#### Lmps per region (spatial distribution)

In [ ]:
carrier_bus = "AC"
locs = n.buses.location[n.buses[n.buses.carrier == carrier_bus].index]
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier == carrier_bus].index].mean()
df = pd.concat([lmps, locs], axis=1).rename(columns={0: f"{carrier_bus}_lmp"})
lmp_regions_onshore = df_regions_onshore[["geometry", "coords", "name"]].copy().rename(columns={"name": "location"})
lmp_regions_onshore = pd.merge(lmp_regions_onshore,df,on="location").set_index("location")
lmp_regions_onshore.head(3)

In [ ]:
fig = plt.figure(figsize=(12, 8))
crs = ccrs.EqualEarth()

ax = plt.axes(projection=ccrs.EqualEarth())
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')
ax.set_title(f"{carrier_bus}_lmp (locational marginal price)", fontsize=16, **font1)

lmp_regions_onshore.to_crs(crs.proj4_init).plot(column=f"{carrier_bus}_lmp",
                     ax=ax,
                     cmap=plt.get_cmap("magma_r"),
                     linewidth=0.05,
                     edgecolor = 'grey',
                     legend=True,
                     legend_kwds={'label':"€/MWh",
                        'orientation': "vertical"})

plt.show()

#### Nodal Prices for every time step (sum) - heatmap (temporal distribution)

In [ ]:
carrier = "H2" # "battery" # "Li ion" # H2, services rural heat, low voltage (no values for VRE as they do not have their own buses)

# change to mean for investigating in mean (unit: €/MWh)
df = pd.DataFrame(n.buses_t.marginal_price[n.buses[n.buses.carrier==carrier].index].mean(axis=1))

hours = df.index.hour.unique()
df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

for hour in hours:
    df_start[str(hour)] = df[df.index.hour==hour].values

plt.figure(figsize=(12, 4))
ax = sns.heatmap(df_start.transpose(),
                 cmap=plt.get_cmap("magma_r"),
                 linewidth=0.001,
                 xticklabels=15)
plt.title(f"{carrier} lmp (€/MWh)", fontsize=16, **font1)
plt.ylabel("hour of the day", fontsize=12, **font1)
plt.xlabel("day of the year", fontsize=12, **font1)

# Rewrite the y labels
x_labels = ax.get_xticks()
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

plt.show()

__Findings:__
- __H2:__ no clear pattern during the day; seasonal pattern as higher lmps in the winter with increasing lpms during winter from Nov-Jan; rather constant during summer
- __battery:__ cheaper from hour 6 to 12; higher lmps in winter (Nov-Mar) with highest in Jan; rather constant during summer with some more expensive days
- __Li ion:__ same as battery (also same magnitude)
- __low voltage:__ similar to battery with a hgher magnitude in range(0,700)
- __services rural heat:__ range(0,900); most expensive in hours 3,6 and 18; very low values during warmer months (Apr-Oct); highest lmps in Jan
- __urban central heat:__ range(0,130); clear daily pattern: lower in night (hours: 21,0) and during midday (hours: 9,12) and higher in the morning and afternoon; lower in warmer months (Apr-Sep) and higest lmps in winter (Nov-Mar)

##### Case Study: H2 Comparison
comparison of wind driven baltic sea regions and sun driven Mediterranean Sea regions

In [ ]:
help = df_regions_onshore[df_regions_onshore["H2 Electrolysis_gen"] > np.quantile(df_regions_onshore["H2 Electrolysis_gen"], 0.5)][["name", "coords"]]

names = help.name.tolist()

help['x'] = [x[0] for x in help.coords]
help['y'] = [x[1] for x in help.coords]
x = help.x
y = help.y

names = help.name.tolist()

fig, ax = plt.subplots(figsize=(18, 8))
ax.scatter(x, y)

for i, txt in enumerate(names):
    ax.annotate(txt, (x[i], y[i]))

In [ ]:
north = help[help['y'] >= 45].index
south = help[help['y'] < 45].index

In [ ]:
carrier = "H2" # "battery" # "Li ion" # H2, services rural heat, low voltage (no values for VRE as they do not have their own buses)
bus_index = south

buses = n.buses[n.buses.carrier==carrier].index
help_df = pd.DataFrame(n.buses.loc[buses]["location"])
help_df["bus"] = help_df.index
help_df.set_index('location', inplace=True)

# change to mean for investigating in mean (unit: €/MWh)
df = pd.DataFrame(n.buses_t.marginal_price[pd.Index(help_df.loc[bus_index].bus)].mean(axis=1))

hours = df.index.hour.unique()
df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

for hour in hours:
    df_start[str(hour)] = df[df.index.hour==hour].values

plt.figure(figsize=(12, 4))
ax = sns.heatmap(df_start.transpose(),
                 cmap=plt.get_cmap("magma_r"),
                 linewidth=0.001,
                 xticklabels=15)
plt.title(str(carrier), fontsize=16, **font1)
plt.ylabel("hour of the day", fontsize=12, **font1)
plt.xlabel("day of the year", fontsize=12, **font1)

# Rewrite the y labels
x_labels = ax.get_xticks()
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

plt.show()

#### Seasonal Pattern of nodal Prices (temporal distribution)

In [ ]:
# # Plot of lmp daily pattern of lmps over all carriers normalized by the mean price of the current day
carrier_bus = "H2" #carriers_buses
locs = n.buses.location[n.buses[n.buses.carrier == carrier_bus].index]
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier == carrier_bus].index]

plt.figure(figsize=(12, 8))
df3 = pd.DataFrame(lmps.mean(axis=1))
df3.columns = ["lmp"]
df3["lmp_normalized"] = np.nan

for snap in df3.index:
    df3.loc[snap, "lmp_normalized"] = df3.loc[snap, "lmp"] / df3[df3.index.date == snap.date()].lmp.mean()

df3["hour_of_day"] = df3.index.hour
sns.boxplot(data=df3, y="lmp_normalized", x="hour_of_day")
plt.title(f"{carrier_bus} lmps for all carriers and time steps normalized by the mean price of the current day")
plt.show()

#### GIF

##### days

carrier = "onwind"
df = market_values_by_time_index(n, days, carrier)
df = df.apply(pd.to_numeric)

for day in days:

    fig = plt.figure(figsize=(10, 8))

    crs = ccrs.AlbersEqualArea()
    ax = plt.axes(projection=ccrs.AlbersEqualArea())
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')
    ax.set_title(f"{carrier} market value @ {day.date()} in €/MWh", fontsize=22, **font1)

    df2 = onshore_regions.copy()
    df2[carrier] = df.loc[day]

    df2.to_crs(crs.proj4_init).plot(column=carrier,
                         ax=ax,
                         cmap=plt.get_cmap("magma_r"),
                         linewidth=0.05,
                         edgecolor = 'grey',
                         legend=True,
                         legend_kwds={'orientation': "vertical",
                            'shrink' : 1}
                                    )
    fig.tight_layout()
    fig.savefig(f"{PLOT_DIR}\GIF\onwind_mv\days\mv_onwind_{day.date()}.png")
    plt.close()

##### weeks

carrier = "onwind"
weeks = np.arange(1,53,1)

mvs=[]
for week in weeks:
    gen = n.generators_t.p.loc[n.generators_t.p.index.week == week, n.generators.carrier == carrier]
    gen.columns = gen.columns.map(n.generators.bus)
    lmp = n.buses_t.marginal_price.loc[n.buses_t.marginal_price.index.week == week, gen.columns]
    mv = (gen * lmp).sum() / gen.sum()
    mvs.extend(mv.values)

vmin = np.nanmin(mvs)
vmax = np.nanmax(mvs)

for week in weeks:

    fig = plt.figure(figsize=(10, 8))

    crs = ccrs.AlbersEqualArea()
    ax = plt.axes(projection=ccrs.AlbersEqualArea())
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    if week < 10:
        week_title = f"0{week}"
    else:
        week_title = week

    ax.set_title(f"{carrier} market value @ week {week_title} (from {n.generators_t.p[n.generators_t.p.index.week == week].index[0].date().strftime('%b %d')}) in €/MWh", fontsize=18, **font1)

    # calc mv
    gen = n.generators_t.p.loc[n.generators_t.p.index.week == week, n.generators.carrier == carrier]
    gen.columns = gen.columns.map(n.generators.bus)
    lmp = n.buses_t.marginal_price.loc[n.buses_t.marginal_price.index.week == week, gen.columns]
    mv = (gen * lmp).sum() / gen.sum()
    mv.index = mv.index.map(n.buses.location)
    df = onshore_regions.copy()
    df[carrier] = mv

    df.to_crs(crs.proj4_init).plot(column=carrier,
                         ax=ax,
                         cmap=plt.get_cmap("magma_r"),
                         vmin=vmin,
                         vmax=vmax,
                         linewidth=0.05,
                         edgecolor = 'grey',
                         legend=True,
                         legend_kwds={'orientation': "vertical",
                            'shrink' : 1}
                                    )
    fig.tight_layout()
    fig.savefig(f"{PLOT_DIR}\GIF\onwind_mv\weeks\mv_onwind_week{week}.png")
    plt.close()

### Time series analysis

- when do different technologies produce?
- when are their revenues created? (nodal price * production)
- what are the prices when they produce?


#### Technology Operation and Revenue by Price band

##### Energy conversion technologies production and revenue by output technology price band

In [ ]:
# plot percentage of production per price band and percentage of revenue per price band
carriers_gen = ["onwind", "solar", "ror", "offwind-ac", 'urban central solar thermal']
carriers_link = ["OCGT", "H2 Electrolysis", "H2 Fuel Cell", "urban central gas CHP", "SMR", "SMR CC","electricity distribution grid", 'urban central air heat pump', 'urban central water tanks discharger','urban central resistive heater', 'urban central gas boiler', 'battery discharger']
carriers_labels = (carriers_gen + carriers_link)
price_bands = [-np.inf, 0, 5, 50, 100, 200, 1000, np.inf]
price_bands_labels = ["< 0","0 - 5", "5 - 50", "50 - 100", "100 - 200", "200 - 1000", "> 1000"]
res_prod = pd.DataFrame(index=[str(s) for s in range(len(price_bands)-1)], columns=carriers_labels)
res_rev = pd.DataFrame(index=[str(s) for s in range(len(price_bands)-1)], columns=carriers_labels)

In [ ]:
for carrier in carriers_gen:
    gen = n.generators_t.p.loc[:, n.generators.carrier == carrier]
    gen.columns = gen.columns.map(n.generators.bus)
    lmp = n.buses_t.marginal_price.loc[:, gen.columns]
    rev = gen * lmp

    for i in range(len(price_bands)-1):
        lmp_cat = lmp.mask((lmp >= price_bands[i]) & (lmp < price_bands[i+1]), "hit")
        res_prod.loc[str(i), carrier] = gen[lmp_cat == "hit"].sum().sum() / gen.sum().sum()
        res_rev.loc[str(i), carrier] = rev[lmp_cat == "hit"].sum().sum() / rev.sum().sum()

for carrier in carriers_link:
    # prices of the generated energy
    gen = abs(n.links_t.p1.loc[:, n.links.carrier == carrier])
    gen.columns = gen.columns.map(n.links.bus1)
    lmp = n.buses_t.marginal_price.loc[:, gen.columns]
    rev = gen * lmp

    for i in range(len(price_bands)-1):
        lmp_cat = lmp.mask((lmp >= price_bands[i]) & (lmp < price_bands[i+1]), "hit")
        res_prod.loc[str(i), carrier] = gen[lmp_cat == "hit"].sum().sum() / gen.sum().sum()
        res_rev.loc[str(i), carrier] = rev[lmp_cat == "hit"].sum().sum() / rev.sum().sum()

In [ ]:
res_prod

In [ ]:
res_rev

In [ ]:
# plot shows price bands for different technologies (prices of the output technology is used)
# onwind is categorised by the price of electricity and H2 Electrolysis by the price of H2

fig, axes = plt.subplots(2, 1, figsize=(14, 6))

fig.suptitle('Technology operation and revenue by price band')

ax0 = res_prod.transpose().plot(kind='bar', stacked=True, color=sns.color_palette("magma_r", 7), ax=axes[0], legend=False)
ax0.set_ylabel("Delivered Energy by Energy Price")
ax0.set_xlabel("carrier")
ax0.set_xticks(ticks=range(len(carriers_labels)), labels=carriers_labels, rotation=45)

ax1 = res_rev.transpose().plot(kind='bar', stacked=True, color=sns.color_palette("magma_r", 7), ax=axes[1])
ax1.legend(title='Price Band (€/MWh', labels=price_bands_labels, bbox_to_anchor=(1, 1))
ax1.set_ylabel("Revenue by Energy Price")
ax1.set_xlabel("carrier")
ax1.set_xticks(ticks=range(len(carriers_labels)), labels=carriers_labels, rotation=45)

fig.tight_layout()
#plt.close()

__Findings:__
- Implications:
- __onwind:__ 20 % energy delivered in band 1 but almost no impact on revenue; price bands 1-3 account for over 90% of delivered energy but only for 70% of the revenue stream; price bands 4 & 5 account for less than 10 % of delivered energy but for 30 % of revenues
- __solar:__ similar to onwind
- __ror:__ General higher production in more expensive price bands 1-3 account for 70 % of production but only for 40 % of revenue
- __H2 Electrolysis:__ production mostly within price bands 3 & 4 and revenue almost only in price band 3 & 4

##### Energy conversion technologies production and revenue by input technology price band (links)

In [ ]:
carriers = car_link_mix.copy()
carriers.extend(["H2 liquefaction", "electricity distribution grid", "urban central water tanks charger"])

res_prod2 = pd.DataFrame(index=[str(s) for s in range(len(price_bands)-1)], columns=carriers)
res_rev2 = pd.DataFrame(index=[str(s) for s in range(len(price_bands)-1)], columns=carriers)

In [ ]:
for carrier in carriers:
# prices of the consumed energy
    con = n.links_t.p0.loc[:, n.links.carrier == carrier]
    transform_columns = n.links_t.p0.loc[:, n.links.carrier == carrier].columns.map(n.links.bus1)
    con.columns = con.columns.map(n.links.bus0)
    lmp_con = n.buses_t.marginal_price.loc[:, con.columns]
    cost = con * lmp_con

    gen = abs(n.links_t.p1.loc[:, n.links.carrier == carrier])
    gen.columns = gen.columns.map(n.links.bus1)
    lmp_gen = n.buses_t.marginal_price.loc[:, gen.columns]
    rev = gen * lmp_gen

    for i in range(len(price_bands)-1):
        lmp_cat1 = lmp_con.mask((lmp_con >= price_bands[i]) & (lmp_con < price_bands[i+1]), "hit")
        # pay attention to indices (if something went wrong then probably here)
        lmp_cat1.columns = transform_columns
        res_prod2.loc[str(i), carrier] = gen[lmp_cat1 == "hit"].sum().sum() / gen.sum().sum()
        res_rev2.loc[str(i), carrier] = rev[lmp_cat1 == "hit"].sum().sum() / rev.sum().sum()

In [ ]:
res_prod2

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

fig.suptitle('Energy conversion technologies production and revenue by input technology price band (links) ')

ax0 = res_prod2.transpose().plot(kind='bar', stacked=True, color=sns.color_palette("magma_r", 7), ax=axes[0], legend=False)
ax0.set_ylabel("Delivered Energy by Energy Price")
ax0.set_xlabel("carrier")
ax0.set_xticks(ticks=range(len(carriers)), labels=carriers, rotation=45)

ax1 = res_rev2.transpose().plot(kind='bar', stacked=True, color=sns.color_palette("magma_r", 7), ax=axes[1])
ax1.legend(title='Price Band (€/MWh', labels=price_bands_labels, bbox_to_anchor=(1, 1))
ax1.set_ylabel("Revenue by Energy Price")
ax1.set_xlabel("carrier")
ax1.set_xticks(ticks=range(len(carriers)), labels=carriers, rotation=45)

fig.tight_layout()
plt.close()

__Findings:__
- ...

##### Electricity (AC)

In [ ]:
# check which technologies produce and consume which specific type of energy
bus_carrier = "low voltage" #"services urban decentral heat" # "urban central heat" # "services rural heat" # "battery"# "H2 liquid" #"H2" # "low voltage" # "AC"
# buses that have the specifiy type of energy as their carrier
buses = n.buses[n.buses.carrier == bus_carrier].index
# generators whch are at the bus
print(f'generators (producer): {n.generators[n.generators.bus.isin(buses)].carrier.unique().tolist()} \n')
print(f'links (consumer):  {n.links[n.links.bus0.isin(buses)].carrier.unique()} \n')
print(f'links (producer): {n.links[n.links.bus1.isin(buses)].carrier.unique()}')

In [ ]:
# is it relevant which type of electricity is compared? AC, low voltage, -> yes

el_cons = ['H2 Electrolysis', 'battery charger' ]
el_prods_gens = ['onwind', 'offwind-ac', 'solar', 'ror']
el_prods_links = ['OCGT', 'H2 Fuel Cell', 'battery discharger']
carriers = el_cons + el_prods_gens + el_prods_links

fig, axes = plt.subplots(2, 1, figsize=(14, 6))

fig.suptitle('Electricity (AC): Production and revenue by electricity price')

prod = pd.concat([res_prod2[el_cons], res_prod[el_prods_gens + el_prods_links]])
rev = pd.concat([res_rev2[el_cons], res_rev[el_prods_gens+el_prods_links]])

ax0 = prod.transpose().plot(kind='bar', stacked=True, color=sns.color_palette("magma_r", 7), ax=axes[0], legend=False)
ax0.set_ylabel("Delivered Energy by Energy Price")
# ax0.set_xlabel("carrier")
ax0.set_xticks(ticks=range(len(carriers)), labels=carriers, rotation=0)

ax1 = rev.transpose().plot(kind='bar', stacked=True, color=sns.color_palette("magma_r", 7), ax=axes[1])
ax1.set_ylabel("Revenue by Energy Price")
ax1.legend(title='Price Band (€/MWh)', labels=price_bands_labels , bbox_to_anchor=(1, 1))
# ax1.set_xlabel("carrier")
ax1.set_xticks(ticks=range(len(carriers)), labels=carriers, rotation=0)

ax1.axvline(x=len(el_cons)-0.5,ymin=-0.15,ymax=2.22,c="red",linewidth=2,linestyle="--", zorder=3, clip_on=False, label="seperator")

fig.tight_layout()

In [ ]:
res_prod2[el_cons]

##### Hydrogen (not liquified)

In [ ]:
h2_cons = ['H2 Fuel Cell', 'Sabatier', 'Fischer-Tropsch', 'H2 liquefaction']
h2_prods = ['H2 Electrolysis', 'SMR', 'SMR CC']
carriers = h2_cons + h2_prods

fig, axes = plt.subplots(2, 1, figsize=(14, 6))

fig.suptitle('Hydrogen: Production and revenue by hydrogen price')

prod = pd.concat([res_prod2[h2_cons], res_prod[h2_prods]])
rev = pd.concat([res_rev2[h2_cons], res_rev[h2_prods]])

ax0 = prod.transpose().plot(kind='bar', stacked=True, color=sns.color_palette("magma_r", 7), ax=axes[0], legend=False)
ax0.set_ylabel("Delivered Energy by Energy Price")
# ax0.set_xlabel("carrier")
ax0.set_xticks(ticks=range(len(carriers)), labels=carriers, rotation=0)

ax1 = rev.transpose().plot(kind='bar', stacked=True, color=sns.color_palette("magma_r", 7), ax=axes[1])
ax1.set_ylabel("Revenue by Energy Price")
ax1.legend(title='Price Band (€/MWh)', labels=price_bands_labels , bbox_to_anchor=(1, 1))
# ax1.set_xlabel("carrier")
ax1.set_xticks(ticks=range(len(carriers)), labels=carriers, rotation=0)

ax1.axvline(x=len(h2_cons)-0.5,ymin=-0.15,ymax=2.22,c="red",linewidth=2,linestyle="--", zorder=3, clip_on=False, label="seperator")

fig.tight_layout()

##### Heat (urban central heat)

In [ ]:
# urban central heat

he_cons = ['urban central water tanks charger' ]
he_prods_gens = ['urban central solar thermal']
he_prods_links = ['urban central air heat pump', 'urban central water tanks discharger','urban central resistive heater', 'urban central gas boiler']
carriers = he_cons + he_prods_gens + he_prods_links
# make newline after 2 words of label
labels = [' '.join(carrier.split(' ')[:2] + ["\n"] + carrier.split(' ')[2:]) for carrier in carriers]

fig, axes = plt.subplots(2, 1, figsize=(14, 6))

fig.suptitle('Heat (urban central heat): Production and revenue by heat price')

prod = pd.concat([res_prod2[he_cons], res_prod[he_prods_gens+he_prods_links]])
rev = pd.concat([res_rev2[he_cons], res_rev[he_prods_gens+he_prods_links]])

ax0 = prod.transpose().plot(kind='bar', stacked=True, color=sns.color_palette("magma_r", 7), ax=axes[0], legend=False)
ax0.set_ylabel("Delivered Energy by Energy Price")
# ax0.set_xlabel("carrier")
ax0.set_xticks(ticks=range(len(carriers)), labels=labels, rotation=0)

ax1 = rev.transpose().plot(kind='bar', stacked=True, color=sns.color_palette("magma_r", 7), ax=axes[1])
ax1.set_ylabel("Revenue by Energy Price")
ax1.legend(title='Price Band (€/MWh)', labels=price_bands_labels , bbox_to_anchor=(1, 1))
# ax1.set_xlabel("carrier")
ax1.set_xticks(ticks=range(len(carriers)), labels=labels , rotation=0)

ax1.axvline(x=len(he_cons)-0.5,ymin=-0.2,ymax=2.3,c="red",linewidth=2,linestyle="--", zorder=3, clip_on=False, label="seperator")

fig.tight_layout()

#### Seasonality Analysis: Duration Curve, histplot, Fourier Transform

- problem1: you only have data for one year
- problem2: data is in 3h steps
-
without further adjustments:
- daily pattern would be after 8 snaps (8*3=24) as we have a 3-hourly dataset
- weekly after 56 snaps
- monthly pattern after 240
- yearly pattern cannot be observed as we only have data for one year
with adjustments
- daily: 24
- weekly: 168
- monthly: 5040

##### Generation

In [ ]:
# get generation of all carriers
gen = pd.DataFrame(index=n.generators_t.p.index)

for carrier in n.generators.carrier.unique():
    gen[carrier]= n.generators_t.p.loc[:, n.generators.carrier == carrier].mean(axis=1)

for carrier in n.links.carrier.unique():
    gen[carrier]= abs(n.links_t.p1.loc[:, n.links.carrier == carrier]).mean(axis=1)

for carrier in n.storage_units.carrier.unique():
    gen[carrier]= n.storage_units_t.p.loc[:, n.storage_units.carrier == carrier].mean(axis=1)

carriers = car_imp_fneum
gen_investigate = gen[carriers]
gen_investigate.head()

###### yearly plot

In [ ]:
# yearly plot
for c in gen_investigate.columns:
    fig, ax = plt.subplots()
    gen_investigate[c].plot(ax=ax, ylabel=c)

###### monthly plot

In [ ]:
# monthly plot
for c in gen_investigate.columns:
    fig, ax = plt.subplots()
    gen_investigate.loc["2013-04", c].plot(ax=ax, ylabel=c)

###### patterns ("8h", "D", "W", "M")
- 8h resamples every 8 hours: 2013-01-01 00:00:00, 2013-01-01 08:00:00, 2013-01-01 16:00:00, 2013-01-02 00:00:00
- D: daily mean; W: weekly mean, M: monthly mean

In [ ]:
# patterns
for c in gen_investigate.columns:
    fig, ax = plt.subplots()
    for res in ["8h", "D", "W", "M"]:
        gen_investigate[c].resample(res).mean().plot(ax=ax, label=res)
    ax.set_ylabel(c)
    plt.legend()

###### duration curve

In [ ]:
for c in gen_investigate.columns:
    fig, ax = plt.subplots()
    duration_curve = gen_investigate[c].sort_values(ascending=False).reset_index(drop=True)
    duration_curve.plot(ax=ax, ylabel=c, xlabel="hours of the year")

###### histplot

In [ ]:
for c in gen_investigate.columns:
    fig, ax = plt.subplots()
    gen_investigate[c].plot.hist(bins=20, ax=ax)
    ax.set_xlabel(c)

###### furier transform

In [ ]:
# fourier transform
for c, d in gen_investigate.iteritems():
    fig, ax = plt.subplots()
    fft = abs(pd.Series(np.fft.rfft(d - d.mean()), index=np.fft.rfftfreq(len(d), d=1./2920))**2)
    fft.plot(xlim=[0,1000], ylabel=f"Power Spectral Density ({c})", xlabel="Frequency (1/year)")

###### correlation

In [ ]:
plt.figure(figsize=(14, 8))
sns.heatmap(gen_investigate.corr(), annot=True)
plt.show()

##### Lmps

In [ ]:
# get lmps of all buses
lmp = pd.DataFrame(index=n.buses_t.marginal_price.index)

for carrier in n.buses.carrier.unique():
    lmp[carrier]= n.buses_t.marginal_price.loc[:, n.buses.carrier == carrier].mean(axis=1)

carriers = car_bus_imp
lmp_investigate = lmp[carriers]
lmp_investigate.head()

###### yearly plot

In [ ]:
# yearly plot
for c in lmp_investigate.columns:
    fig, ax = plt.subplots()
    lmp_investigate[c].plot(ax=ax, ylabel=c)

###### monthly plot

In [ ]:
for c in lmp_investigate.columns:
    fig, ax = plt.subplots()
    lmp_investigate.loc["2013-04", c].plot(ax=ax, ylabel=c)

###### patterns

In [ ]:
for c in lmp_investigate.columns:
    fig, ax = plt.subplots()
    for res in ["8h", "D", "W", "M"]:
        lmp_investigate[c].resample(res).mean().plot(ax=ax, label=res)
    ax.set_ylabel(c)
    plt.legend()

###### price duration curve

In [ ]:
for c in lmp_investigate.columns:
    fig, ax = plt.subplots()
    duration_curve = lmp_investigate[c].sort_values(ascending=False).reset_index(drop=True)
    duration_curve.plot(ax=ax, ylabel=c, xlabel="hours of the year")

In [ ]:
colors = matplotlib.cm.tab20(range(len(carriers)))

for c, i  in zip(lmp_investigate.columns, np.arange(1,len(carriers))):
    duration_curve = lmp_investigate[c].sort_values(ascending=False).reset_index(drop=True)
    plt.plot(duration_curve, label=c, color=colors[i])

plt.ylim([0,1000])
plt.xlim([0,1000])
plt.legend()
plt.show()

###### histplot

In [ ]:
for c in lmp_investigate.columns:
    fig, ax = plt.subplots()
    lmp_investigate[c].plot.hist(bins=20, ax=ax)
    ax.set_xlabel(c)

###### fourier transform
Explanantion:
- Frequency are going from seasonal patterns (closer to 0) to daily (x=365) and to within daily patterns (x > 365)
- high value at x = 1 means that there is a pattern that occurs once over the whole time series (yearly pattern, seasonal pattern
- high value at x = 365 means that there is a pattern that is occuring 365 times per year which means it occurs daily; this is typically for solar as the rythm of the sun is daily
- high value at x = 730 means there is a pettern that occurs 730 times per year which is two times per day; this is for example for heating the case where ther eis a peak in the morning adn in the afternoon

In [ ]:
for c, d in lmp_investigate.iteritems():
    fig, ax = plt.subplots()
    fft = abs(pd.Series(np.fft.rfft(d - d.mean()), index=np.fft.rfftfreq(len(d), d=1./2920))**2)
    fft.plot(xlim=[0,1000], ylabel=f"Power Spectral Density ({c})", xlabel="Frequency (1/year)")

# North south separation do detect what determines the pattern

###### correlation

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(lmp_investigate.corr(), annot=True)
plt.show()

### Congestion rent
Additionally: Calculate congestion rents for link components: price difference i-j * flow i-j
ToDo: maybe also for line components?

In [ ]:
# congestion rent makes only sense if link has only 2 buses; definition of cr is otherwise unclear
carriers = n.links.carrier[n.links.bus3 == ""].unique()
cr_res1 = pd.DataFrame(index = range(1), columns=carriers)

for carrier in carriers:
    cr_res1[carrier] = (congestion_rent_link(n,carrier).sum().sum() / 1000000)

In [ ]:
cr_res1 = cr_res1.sort_values(by=0, axis='columns', ascending=False)

In [ ]:
sns.set(rc={'figure.figsize':(20,5)})
sns.barplot(data=cr_res1)
ticks = plt.xticks(rotation=90)
plt.title("congestion rent in million € per year")
plt.show()

__Findings__
- negative values for some technologies: OCGT, water tanks, battery charger but in much smaller magnitude than positive values (maybe due to numeric inaccuracy?)

In [ ]:
# Congestion rent by region
cr_regions_onshore = onshore_regions.copy().rename(columns={"name": "location"})

for carrier in carriers:
    cr_regions = congestion_rent_link(n,carrier).sum()

    if n.buses.loc[cr_regions.index.map(n.links.bus0)].location.values[0] == 'EU':
        bus = "bus1"
    else:
        bus = "bus0"

    cr_regions = pd.concat([cr_regions,n.links.loc[cr_regions.index][bus]], axis=1).rename(columns={0: f"{carrier}_cr"})
    cr_regions["location"] = n.buses.loc[cr_regions.index.map(n.links[bus])].location.values
    # for some carrier e.g. "DC" the locations occur several times in the index -> take sum
    cr_regions = cr_regions.groupby(by=["location"], axis="index").sum()
    # convert to million € per year
    cr_regions_onshore[f"{carrier}_cr"] = (cr_regions[f"{carrier}_cr"] / 1000000)
cr_regions_onshore.head(3)
    # TODO: For plotting the cr into a map which bus should be used? bus0, where the energy comes from or bus1 where the energy goes to? For some carriers e.g. Fischer-Tropsch they are not in the same region (EU bus)

In [ ]:
carrier = "H2 Electrolysis"

fig = plt.figure(figsize=(10, 8))

crs = ccrs.EqualEarth()
ax = plt.axes(projection=ccrs.EqualEarth()) # Mercator
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')
ax.set_title(f"{carrier} congestion rent", fontsize=16, **font1)

cr_regions_onshore.to_crs(crs.proj4_init).plot(column=f"{carrier}_cr",
                     ax=ax,
                     cmap=plt.get_cmap("magma_r"),
                     linewidth=0.05,
                     edgecolor = 'grey',
                     legend=True,
                     legend_kwds={'label':"million € per year",
                        'orientation': "vertical",
                                  'shrink' : 0.8})

plt.show()
# TODO: Would be interesting to plot a map with the location of the capacity of the corresponding carrier next to it; probably there is a correlation between higher capacity and higher cr

In [ ]:
cr_regions_onshore[f"{carrier}_cr"].sort_values(ascending=False)

__Findings:__
- __H2 Electrolysis:__  Highest values in IRL, GBR and DNK; General higher values in the coastal area of mainland Europe (PRT, ESP, FRA, NLD, DEU, POL, GRC); very low values in the inner mainland and scandinavia
- __electricity distribution grid:__ no strong pattern; highest values for north/middle (FR0 1) and GB5 13; there might be soe correlation with higher cr in population-intensive regions an
- __Fischer-Tropsch:__ Comparable to H2 Electrolysis; Highest values in DNK & GBR; especialle higher values on the resource receiving party of the european coastline (Atlantic coast and GRC)
- __urban central solid biomass CHP CC:__ High values for parts of DEU, CZE, AUT, ITA, SVK & HUN; Very low values for west europe UK and scandinavia
- residential rural ground heat pump:
- residential urban decentral air heat pump:
- H2 liquefaction:
- urban central air heat pump:
- urban central gas CHP:

In [ ]:
# summarising plot with mv, gen and cr
carrier="H2 Electrolysis"
# "battery discharger" # "battery charger" # "H2 Fuel Cell" #"Fischer-Tropsch" #"OCGT" #H2 Electrolysis" # H2 Electrolysis, H2 Fuel Cell
carriers=[f"{carrier}_mv" , f"{carrier}_gen", f"{carrier}_cr"]

fig, axs = plt.subplots(ncols=3, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 16))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs):
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    if i==0:
        title = f"{carrier} market value"
        label = "€/MWh"
        df = df_regions_onshore
    elif i==1:
        title = f"{carrier} generation"
        label = "TWh"
        df = df_regions_onshore
    else:
        title = f"{carrier} congestion rent"
        label = "million €"
        df = cr_regions_onshore

    args = dict(column=carriers[i],
                     ax=ax,
                     cmap=plt.get_cmap("magma_r"),
                     linewidth=0.05,
                     edgecolor = 'grey',
                     legend=True,
                     legend_kwds={'label':label,
                        'orientation': "vertical",
                                  'shrink' : 0.3}
                )

    ax.set_title(title, fontsize=16, **font1)
    df.to_crs(crs.proj4_init).plot(**args)

plt.show()

### Two component analysis (Scatter)

#### Scatterplot: market value and generation
scatterplot of the market value and the generation for the 181 regions (if both values are present)

In [ ]:
carrier = "OCGT" # "battery charger" # "OCGT"# "H2 Electrolysis" #"ror" # "onwind" # "solar"
x = df_regions_onshore[f"{carrier}_gen"]
y = df_regions_onshore[f"{carrier}_mv"]
x = x[x.notna() & y.notna()]
y = y[x.notna() & y.notna()]

fig = plt.figure(figsize=(10, 6))
p = np.poly1d(np.polyfit(x, y, 1))
t = np.linspace(start = 0, stop = int(max(x)), num = 1000)
plt.plot(x, y, 'o', label=carrier)
plt.plot(t, p(t), '-')
plt.title(f"Scatter: generation vs. market value ({carrier})")
plt.xlabel("generation (TWh)")
plt.ylabel("market value (€/MWh)")
plt.legend()
plt.show()

__Findings:__
- __solar:__ most generation less than 0.5 e^7; downward trend for the relationship between market value and generation
- __onwind:__ most gen < 2 e^7 MWh; clear downward trend
- __ror:__ mv between 60 and 150 €/MWh; no trend
- __H2 Electrolysis:__ gen mostly < 2 e^7; slight downward trend
- __OCGT:__ mostly very low gen; clear upward trend
- __battery charger:__ no clear trend (slightly downward)
- __SMR:__ most gen < 10000 MWh; clear upward trend

In [ ]:
# comparing plot:
carriers = car_gen_vre.copy() + car_link_imp1.copy()
carriers = [e for e in carriers if e not in ["Fischer-Tropsch", "Sabatier"]]
colors = matplotlib.cm.tab20(range(len(carriers)))

plt.figure(figsize=(10, 6))

for carrier, i in zip(carriers, np.arange(0,len(carriers)-1)):
    x = df_regions_onshore[f"{carrier}_gen"]
    y = df_regions_onshore[f"{carrier}_mv"]
    x = x[x.notna() & y.notna()]
    y = y[x.notna() & y.notna()]

    p = np.poly1d(np.polyfit(x, y, 1))
    t = np.linspace(start = 0, stop = int(max(x)), num = 5)
    plt.plot(t, p(t), '--', label=carrier, color=colors[i], marker=i)

plt.title("Trend: Generation vs. market value")
plt.xlabel("Generation (TWh)")
plt.ylabel("market value (€/MWh)")
#plt.ylim([0,600])
plt.xlim([0,2*10**4])
plt.legend()
plt.show()

#### Scatterplot: market value and full load hours

In [ ]:
th = 0.9

# get full_load hours (generators)
for carrier in n.generators.carrier.unique():
    gen = n.generators_t.p[n.generators[n.generators.carrier == carrier].index]
    # max_output = n.generators.p_nom_opt[n.generators.carrier == carrier]
    max_output = gen.max()

    # ts with p_nom_opt
    max_output_ts = gen.copy()
    for snap in n.storage_units_t.p.index:
        max_output_ts.loc[snap] = max_output[max_output_ts.columns]

    # calc full load hours
    flh = (gen >= th * max_output_ts).sum()
    flh.index = flh.index.map(n.generators.bus).map(n.buses.location)
    df_regions_onshore[f"{carrier}_flh"] = flh

# links
for carrier in n.links.carrier.unique():
    gen = abs(n.links_t.p1.loc[:, n.links.carrier == carrier])
    # use p_nom_opt as indicator for full load hour
    # max_output = n.links.p_nom_opt[n.links.carrier == carrier]
    # use maximum of generation output as indicator for full load hour
    max_output = gen.max()

    # ts with p_nom_opt
    max_output_ts = gen.copy()
    for snap in n.storage_units_t.p.index:
        max_output_ts.loc[snap] = max_output[max_output_ts.columns]

    # calc full load hours
    flh = (gen >= th * max_output_ts).sum()
    flh.index = flh.index.map(n.links.bus1).map(n.buses.location)
    # group duplicate index entries
    flh = flh.groupby(by=["Link"], axis="index").sum()
    df_regions_onshore[f"{carrier}_flh"] = flh

# storage units
for carrier in n.storage_units.carrier.unique():
    gen = generation_storage_units(n, carrier)
    max_output = gen.max()

        # ts with p_nom_opt
    max_output_ts = gen.copy()
    for snap in n.storage_units_t.p.index:
        max_output_ts.loc[snap] = max_output[max_output_ts.columns]

    # calc full load hours
    flh = (gen >= th * max_output_ts).sum()
    flh.index = flh.index.map(n.links.bus1).map(n.buses.location)
    # group duplicate index entries
    flh = flh.groupby(by=["Link"], axis="index").sum()
    df_regions_onshore[f"{carrier}_flh"] = flh

df_regions_onshore.head(3)

In [ ]:
n.storage_units_t.p.index

In [ ]:
carrier = "OCGT"# "Fischer-Tropsch" # "OCGT" # "onwind" # "solar rooftop" # "solar"# "H2 Electrolysis" # "solar rooftop" # "ror"
x = df_regions_onshore[f"{carrier}_flh"]
y = df_regions_onshore[f"{carrier}_mv"]
x = x[x.notna() & y.notna()]
y = y[x.notna() & y.notna()]

fig = plt.figure(figsize=(10, 6))
p = np.poly1d(np.polyfit(x, y, 1))
t = np.linspace(start = 0, stop = int(max(x)), num = 1000)
plt.plot(x, y, 'o', label=carrier)
plt.plot(t, p(t), '-')
plt.title("Scatter: full load hours vs. market value")
plt.xlabel("full load hours (2920 snaps per year)")
plt.ylabel("market value (€/MWh)")
plt.legend()
plt.show()

__Findings:__
- with p_nom_opt the full load hours for many carriers are 0; maybe it is more valid to use the maximum output as the maximum capacity to compare to?
- for VRE the trend is mostly downwards, for technologies that can be regulated it is upwards
- __ror:__ flh either < 400 or around 1200; upward trend
- __solar / solar rooftop:__ flh > 150; downward trend
- __H2 Electrolysis:__ flh mostly in range(500,1000); upward trend
- __onwind:__ flh mostly < 100 and all < 500; clear downward trend
- __OCGT:__ flh in range (20,60); clear downward trend
- __Fischer-Tropsch:__ no single full load hour

In [ ]:
# comparing plot:
carriers = car_gen_vre.copy() + car_link_imp1.copy()
carriers = [e for e in carriers if e not in ["Fischer-Tropsch", "Sabatier"]]
colors = matplotlib.cm.tab20(range(len(carriers)))

plt.figure(figsize=(10, 6))

for carrier, i in zip(carriers, np.arange(0,len(carriers)-1)):
    x = df_regions_onshore[f"{carrier}_flh"]
    y = df_regions_onshore[f"{carrier}_mv"]
    x = x[x.notna() & y.notna()]
    y = y[x.notna() & y.notna()]

    p = np.poly1d(np.polyfit(x, y, 1))
    t = np.linspace(start = 0, stop = int(max(x)), num = 5)
    plt.plot(t, p(t), '--', label=carrier, color=colors[i], marker=i)

plt.title("Trend: full load hours vs. market value")
plt.xlabel("full load hours (2920 snaps per year)")
plt.ylabel("market value (€/MWh)")
plt.ylim([0,600])
plt.xlim([0,1500])
plt.legend()
plt.show()

__Findings:__
- strong downward trend for OCGT and urban central gas CHP
- strong upward trend for SMR and H2 Fuel Cell
- rather downward: solar, onwind, solar rooftop, offwind-ac, offwind-dc
- rather upward: ror, H2 Electrolysis, battery charger

__Implications:__
- non controllable technologies like vre cannot choos to run during times of high market values, therefore the mv rater decreases with more flh
- controllable energies like SMR, ror, H2 Electrolysis, battery charger can choose when to run adn therefore are more likely to run during times of high mv
- the strong downward trend of OCGT and central gas CHP (both controllable) can be explained by the fact that they are only running at very high mv already

### ToDo

In [ ]:
# plot loads
n.loads.carrier.unique()

In [ ]:
#TODO: market value aggregated over carriers per country/region plot#